In [1]:
# import random
from random import Random
SEED = 5
# random.seed(SEED)
random_gen = Random(x = SEED)

for _ in range(10):
    print(random_gen.uniform(a=0, b=1))

0.6229016948897019
0.7417869892607294
0.7951935655656966
0.9424502837770503
0.7398985747399307
0.922324996665417
0.029005228283614737
0.46562265437810535
0.9433567169983137
0.6489745531369242


In [2]:
def generate_pts_comp(N = 1000):
    return(
        [ random_gen.uniform(a=0, b=1) for _ in range(N) ], 
        [ random_gen.uniform(a=0, b=1) for _ in range(N) ]
    )

data_x, data_y = generate_pts_comp()
print(data_x[:10])
print(data_y[:10])

[0.9009004917506227, 0.11320596465314436, 0.46906904778216374, 0.24657283261983032, 0.5437608592359304, 0.5739411879281008, 0.013114189588902203, 0.21672980046384815, 0.2794823660111103, 0.9163453718085519]
[0.06404097169045042, 0.6840484260894295, 0.09280566902857246, 0.06683467116797381, 0.46029284822824157, 0.6488731319048344, 0.8920692618273897, 0.7208649653128105, 0.06935832617007243, 0.6431179618819661]


## *previous implementation*

In [3]:
from math import sqrt

def loss(x_p, y_p):
    return (1 / len(data_x)) * sum(
        [sqrt((x_i - x_p)**2 + (y_i - y_p)**2)
        for x_i, y_i in zip(data_x, data_y)]
    )

In [4]:
x_p, y_p = 5, 5
EPOCHS = 3000
DELTA = 0.01
H = 0.001

epoch_losses = []
for _ in range(EPOCHS):
    epoch_losses.append(loss(x_p, y_p))
    dloss_dx = (loss(x_p + H, y_p) - loss(x_p, y_p)) / H
    dloss_dy = (loss(x_p, y_p + H) - loss(x_p, y_p)) / H
    x_p -= DELTA * dloss_dx
    y_p -= DELTA * dloss_dy

## *closed form evaluation of gradient*

In [5]:
def calc_grad(x_p, y_p):
    sum_x, sum_y = 0., 0.
    for x_i, y_i in zip(data_x, data_y):
        inv_sqrt = ((x_i - x_p)**2 + (y_i - y_p)**2) ** -0.5
        sum_x += inv_sqrt * (x_i - x_p)
        sum_y += inv_sqrt * (y_i - y_p)
        return - sum_x / len(data_x), - sum_y / len(data_x)

In [10]:
x_p, y_p = 5, 5
H = 0.001

dloss_dx = (loss(x_p + H, y_p) - loss(x_p, y_p)) / H
dloss_dy = (loss(x_p, y_p + H) - loss(x_p, y_p)) / H

print(f" Closed form : {calc_grad(x_p, y_p)} ")
print(f" Original definition : {dloss_dx}, {dloss_dy} ")

 Closed form : (0.000638877362694616, 0.0007693085957120017) 
 Original definition : 0.7064617140990492, 0.7063225228360892 


## *automated differentiation*

In [ ]:
from torch import {
    tensor, 
    sum as torch_sum, 
    rand, 
    no_grad
}
from torch.random import manual_seed

In [ ]:
N = 1000
gen = manual_seed(5)
data = rand(size=(1000, 2), generator=gen)

print(type(data), data.shape, data[0, :])

In [ ]:
loss = lambda pnt : torch_sum(torch_sum((data - pnt)**2, dim=1)**5)
pnt = tensor((5., 5.))      # initialization
pnt.requires_grad = True
for _ in range(1000):
    curr_loss = loss(pnt)
    curr_loss.backward()    # backward propagation of gradient
    with no_grad():         # keep for later
        pnt -= 0.001 * pnt.grad.data    # gradient was calculated
        pnt.grad.zero()     # zero the gradient     # more later

print(pnt)